In [1]:
import cv2
import os

def extract_and_save_keyframes(video_path, output_dir, interval=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    keyframe_count = 0

    # Tạo thư mục lưu trữ khung hình nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)

    success, frame = cap.read()
    
    while success:
        if frame_count % interval == 0:  # Trích xuất mỗi 'interval' khung hình
            keyframe_filename = os.path.join(output_dir, f'keyframe_{keyframe_count:04d}.jpg')
            cv2.imwrite(keyframe_filename, frame)
            keyframe_count += 1
            print(f"Đã lưu: {keyframe_filename}")
        
        success, frame = cap.read()
        frame_count += 1

    cap.release()

# Ví dụ sử dụng
video_path = '/home/thiendc/projects/video_summarization/scene_based/output_video.mp4'
output_dir = 'data_based_scene_keyframe'

extract_and_save_keyframes(video_path, output_dir, interval=30)

Đã lưu: data_based_scene_keyframe/keyframe_0000.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0001.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0002.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0003.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0004.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0005.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0006.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0007.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0008.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0009.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0010.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0011.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0012.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0013.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0014.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0015.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0016.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0017.jpg
Đã lưu: data_based_scene_keyframe/keyframe_0018.jpg
Đã lưu: data

In [2]:
video_path = '../data/sample_tv360/gt/chunk_1_gt.mp4'
output_dir = 'gt_keyframe'
extract_and_save_keyframes(video_path, output_dir, interval=30)

Đã lưu: gt_keyframe/keyframe_0000.jpg
Đã lưu: gt_keyframe/keyframe_0001.jpg
Đã lưu: gt_keyframe/keyframe_0002.jpg
Đã lưu: gt_keyframe/keyframe_0003.jpg
Đã lưu: gt_keyframe/keyframe_0004.jpg
Đã lưu: gt_keyframe/keyframe_0005.jpg
Đã lưu: gt_keyframe/keyframe_0006.jpg
Đã lưu: gt_keyframe/keyframe_0007.jpg
Đã lưu: gt_keyframe/keyframe_0008.jpg
Đã lưu: gt_keyframe/keyframe_0009.jpg
Đã lưu: gt_keyframe/keyframe_0010.jpg
Đã lưu: gt_keyframe/keyframe_0011.jpg
Đã lưu: gt_keyframe/keyframe_0012.jpg
Đã lưu: gt_keyframe/keyframe_0013.jpg
Đã lưu: gt_keyframe/keyframe_0014.jpg
Đã lưu: gt_keyframe/keyframe_0015.jpg
Đã lưu: gt_keyframe/keyframe_0016.jpg
Đã lưu: gt_keyframe/keyframe_0017.jpg
Đã lưu: gt_keyframe/keyframe_0018.jpg
Đã lưu: gt_keyframe/keyframe_0019.jpg
Đã lưu: gt_keyframe/keyframe_0020.jpg
Đã lưu: gt_keyframe/keyframe_0021.jpg
Đã lưu: gt_keyframe/keyframe_0022.jpg
Đã lưu: gt_keyframe/keyframe_0023.jpg
Đã lưu: gt_keyframe/keyframe_0024.jpg
Đã lưu: gt_keyframe/keyframe_0025.jpg
Đã lưu: gt_k

In [3]:
import cv2
import os
from skimage.metrics import structural_similarity as ssim

def load_images_from_folder(folder_path):
    images = []
    image_files = sorted(os.listdir(folder_path))
    for filename in image_files:
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def compare_ssim(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(frame1_gray, frame2_gray, full=True)
    return score

from skimage.metrics import structural_similarity as compare_ssim

def count_matching_keyframes(data_folder, gt_folder, threshold=0.9, window_size=5, ssim_win_size=3):
    data_keyframes = load_images_from_folder(data_folder)
    gt_keyframes = load_images_from_folder(gt_folder)

    matching_count = 0
    gt_length = len(gt_keyframes)

    # Duyệt qua từng khung hình trong data_keyframes
    for i, data_frame in enumerate(data_keyframes):
        # Xác định khoảng window xung quanh khung hình hiện tại
        start = max(0, i - window_size)
        end = min(gt_length, i + window_size + 1)

        # So sánh với các khung hình trong window của gt_keyframes
        for j in range(start, end):
            gt_frame = gt_keyframes[j]
            
            # So sánh SSIM với win_size được chỉ định
            ssim_score = compare_ssim(data_frame, gt_frame, win_size=ssim_win_size)

            # Nếu giống nhau trên ngưỡng, tăng matching_count và bỏ qua các khung còn lại trong window
            if ssim_score >= threshold:
                matching_count += 1
                break  # Dừng kiểm tra nếu đã tìm thấy khung hình phù hợp trong window

    return matching_count


# Ví dụ sử dụng
data_folder = '/home/thiendc/projects/video_summarization/scene_based/data_based_scene_keyframe'
gt_folder = '/home/thiendc/projects/video_summarization/scene_based/gt_keyframe'

matching_keyframes = count_matching_keyframes(data_folder, gt_folder, threshold=0.6)

print(f"Số lượng keyframe giống nhau: {matching_keyframes}")

KeyboardInterrupt: 

In [4]:
import numpy as np

def count_matching_keyframes(data_folder, gt_folder, window_size=10):
    data_keyframes = load_images_from_folder(data_folder)
    gt_keyframes = load_images_from_folder(gt_folder)

    matching_count = 0
    gt_length = len(gt_keyframes)

    # Duyệt qua từng khung hình trong data_keyframes
    for i, data_frame in enumerate(data_keyframes):
        # Xác định khoảng window xung quanh khung hình hiện tại
        start = max(0, i - window_size)
        end = min(gt_length, i + window_size + 1)

        # So sánh với các khung hình trong window của gt_keyframes
        for j in range(start, end):
            gt_frame = gt_keyframes[j]

            # So sánh trực tiếp giữa hai mảng dữ liệu khung hình
            if np.array_equal(data_frame, gt_frame):
                matching_count += 1
                break  # Dừng kiểm tra nếu đã tìm thấy khung hình phù hợp trong window

    return matching_count
data_folder = '/home/thiendc/projects/video_summarization/scene_based/data_based_scene_keyframe'
gt_folder = '/home/thiendc/projects/video_summarization/scene_based/gt_keyframe'

matching_keyframes = count_matching_keyframes(data_folder, gt_folder)

print(f"Số lượng keyframe giống nhau: {matching_keyframes}")

Số lượng keyframe giống nhau: 0


In [5]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics.pairwise import cosine_similarity
from imagehash import phash
from PIL import Image

def load_images_from_folder(folder_path):
    images = []
    image_files = sorted(os.listdir(folder_path))
    for filename in image_files:
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def compare_ssim(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(frame1_gray, frame2_gray, full=True)
    return score

def compare_histogram(frame1, frame2):
    hist1 = cv2.calcHist([frame1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([frame2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()
    score = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return score

def compare_sift(frame1, frame2):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(frame1, None)
    kp2, des2 = sift.detectAndCompute(frame2, None)
    if des1 is None or des2 is None:
        return 0
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    return len(matches)


def compare_cosine(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY).flatten().reshape(1, -1)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY).flatten().reshape(1, -1)
    score = cosine_similarity(frame1_gray, frame2_gray)[0][0]
    return score

def compare_phash(frame1, frame2):
    img1 = Image.fromarray(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
    img2 = Image.fromarray(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
    hash1 = phash(img1)
    hash2 = phash(img2)
    score = 1 - (hash1 - hash2) / len(hash1.hash.flatten())
    return score

def count_matching_keyframes(data_folder, gt_folder, threshold=0.9):
    data_keyframes = load_images_from_folder(data_folder)
    gt_keyframes = load_images_from_folder(gt_folder)

    min_length = min(len(data_keyframes), len(gt_keyframes))
    
    total_ssim = 0
    total_hist = 0
    total_sift = 0
    total_cosine = 0
    total_phash = 0

    for i in range(min_length):
        frame1 = data_keyframes[i]
        frame2 = gt_keyframes[i]

        total_ssim += compare_ssim(frame1, frame2)
        total_hist += compare_histogram(frame1, frame2)
        total_sift += compare_sift(frame1, frame2)
        total_cosine += compare_cosine(frame1, frame2)
        total_phash += compare_phash(frame1, frame2)

    average_ssim = total_ssim / min_length
    average_hist = total_hist / min_length
    average_sift = total_sift / min_length
    average_cosine = total_cosine / min_length
    average_phash = total_phash / min_length

    return {
        'SSIM': average_ssim,
        'Histogram': average_hist,
        'SIFT': average_sift,
        'Cosine Similarity': average_cosine,
        'Perceptual Hash': average_phash
    }

# Ví dụ sử dụng
data_folder = 'data_based_scene_keyframe'
gt_folder = 'gt_keyframe'

similarity_scores = count_matching_keyframes(data_folder, gt_folder, threshold = 0.7)

print(f"Độ tương đồng giữa hai tập keyframe:")
print(f"SSIM: {similarity_scores['SSIM']}")
print(f"Histogram Comparison: {similarity_scores['Histogram']}")
print(f"SIFT Matching: {similarity_scores['SIFT']}")
print(f"Cosine Similarity: {similarity_scores['Cosine Similarity']}")
print(f"Perceptual Hash: {similarity_scores['Perceptual Hash']}")


Độ tương đồng giữa hai tập keyframe:
SSIM: 0.5830199909097923
Histogram Comparison: 0.6672825806012995
SIFT Matching: 996.6976744186046
Cosine Similarity: 0.8220094252447344
Perceptual Hash: 0.5817587209302325


In [6]:
from difflib import SequenceMatcher

def frames_to_hashes(frames):
    # Chuyển đổi các khung hình thành chuỗi băm để so sánh
    hashes = []
    for frame in frames:
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        hash_value = phash(img)
        hashes.append(hash_value)
    return hashes

def compare_frames_with_lcs(data_frames, gt_frames):
    data_hashes = frames_to_hashes(data_frames)
    gt_hashes = frames_to_hashes(gt_frames)

    # Tính toán LCS giữa hai chuỗi băm
    matcher = SequenceMatcher(None, data_hashes, gt_hashes)
    lcs_length = matcher.find_longest_match(0, len(data_hashes), 0, len(gt_hashes)).size

    return lcs_length

# Ví dụ sử dụng
data_frames = load_images_from_folder(data_folder)
gt_frames = load_images_from_folder(gt_folder)
lcs_length = compare_frames_with_lcs(data_frames, gt_frames)

print(f"Độ dài của chuỗi con chung dài nhất: {lcs_length}")


Độ dài của chuỗi con chung dài nhất: 1


In [8]:
import cv2
import os
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Giảm độ phân giải của ảnh
def resize_frame(frame, width=640):
    height = int(frame.shape[0] * (width / frame.shape[1]))
    return cv2.resize(frame, (width, height))

# Hàm đọc danh sách các file ảnh keyframes từ một thư mục
def load_keyframes_from_folder(folder_path):
    keyframes = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = resize_frame(img)  # Giảm độ phân giải ảnh
                keyframes.append(img_resized)
    return keyframes

# Hàm so sánh bằng SSIM
def compare_ssim(frame1, frame2, threshold=0.8):
    similarity = ssim(frame1, frame2)
    return similarity >= threshold

# Hàm so sánh bằng SIFT với BFMatcher
def compare_sift(frame1, frame2, min_matches=10):
    sift = cv2.SIFT_create()

    # Tìm đặc trưng và mô tả của cả hai frame
    kp1, des1 = sift.detectAndCompute(frame1, None)
    kp2, des2 = sift.detectAndCompute(frame2, None)

    if des1 is None or des2 is None:
        return False

    # Sử dụng BFMatcher để khớp các đặc trưng
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Lọc các cặp đặc trưng phù hợp
    good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]

    return len(good_matches) >= min_matches

# Hàm song song hóa việc so sánh SSIM và SIFT
def compare_frames(frame1, frame2, ssim_threshold=0.8, min_sift_matches=10):
    is_ssim_match = compare_ssim(frame1, frame2, ssim_threshold)
    is_sift_match = compare_sift(frame1, frame2, min_sift_matches)
    return is_ssim_match, is_sift_match

# Hàm đếm số lượng keyframe matching với tqdm, SSIM và SIFT
def count_matching_keyframes(keyframes1, keyframes2, ssim_threshold=0.8, min_sift_matches=10):
    matching_ssim_count = 0
    matching_sift_count = 0
    matching_both_count = 0

    # Duyệt qua từng keyframe của folder thứ nhất và sử dụng tqdm để hiển thị tiến trình
    for frame1 in tqdm(keyframes1, desc="So sánh keyframes", unit="frame"):
        results = []

        # Sử dụng ThreadPoolExecutor để song song hóa so sánh
        with ThreadPoolExecutor(max_workers= 10) as executor:
            futures = [executor.submit(compare_frames, frame1, frame2, ssim_threshold, min_sift_matches)
                       for frame2 in keyframes2]
            results = [f.result() for f in futures]

        # Tính số lượng matching
        for is_ssim_match, is_sift_match in results:
            if is_ssim_match:
                matching_ssim_count += 1
            if is_sift_match:
                matching_sift_count += 1
            if is_ssim_match and is_sift_match:
                matching_both_count += 1
                break  # Dừng khi đã tìm thấy match cho frame1

    return matching_ssim_count, matching_sift_count, matching_both_count

# Đường dẫn đến hai thư mục chứa keyframes
folder1 = data_folder  # Thay bằng đường dẫn thực tế
folder2 = gt_folder  # Thay bằng đường dẫn thực tế

# Đọc keyframes từ hai thư mục
keyframes1 = load_keyframes_from_folder(folder1)
keyframes2 = load_keyframes_from_folder(folder2)

# Đếm số lượng keyframe matching với SSIM, SIFT và cả hai
matching_ssim, matching_sift, matching_both = count_matching_keyframes(keyframes1, keyframes2)

# In kết quả
print(f"Số lượng keyframe matching chỉ dựa trên SSIM: {matching_ssim}")
print(f"Số lượng keyframe matching chỉ dựa trên SIFT: {matching_sift}")
print(f"Số lượng keyframe matching (SSIM + SIFT): {matching_both}")



So sánh keyframes: 100%|██████████| 180/180 [10:37<00:00,  3.54s/frame]

Số lượng keyframe matching chỉ dựa trên SSIM: 9
Số lượng keyframe matching chỉ dựa trên SIFT: 10782
Số lượng keyframe matching (SSIM + SIFT): 8


In [11]:
import cv2
import os
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Hàm đọc danh sách các file ảnh keyframes từ một thư mục
def load_keyframes_from_folder(folder_path):
    keyframes = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                keyframes.append(img)
    return keyframes

# Hàm so sánh SSIM giữa hai khung hình
def compare_ssim(frame1, frame2, ssim_threshold=0.8):
    similarity = ssim(frame1, frame2)
    return similarity >= ssim_threshold

# Hàm tính exact matching với window size và sử dụng đa luồng
def exact_matching(keyframes1, keyframes2, window_size=5, ssim_threshold=0.5):
    matching_count = 0
    min_length = min(len(keyframes1), len(keyframes2))  # Số lượng keyframe nhỏ hơn

    # Hàm xử lý song song cho từng cặp khung hình
    def compare_window(i):
        frame1 = keyframes1[i]
        start_idx = max(0, i - window_size)
        end_idx = min(min_length, i + window_size + 1)

        for j in range(start_idx, end_idx):
            frame2 = keyframes2[j]
            if compare_ssim(frame1, frame2, ssim_threshold):
                return True  # Tìm thấy khung hình matching

        return False  # Không tìm thấy khung hình matching

    # Sử dụng tqdm để hiển thị tiến trình
    with ThreadPoolExecutor(max_workers= 20) as executor:
        results = list(tqdm(executor.map(compare_window, range(min_length)),
                            desc=f"Window size {window_size}", unit="frame", total=min_length))

    # Đếm số lượng matching
    matching_count = sum(results)

    return matching_count

# Đường dẫn đến hai thư mục chứa keyframes
folder1 = data_folder  # Thay bằng đường dẫn thực tế
folder2 = gt_folder  # Thay bằng đường dẫn thực tế

# Đọc keyframes từ hai thư mục
keyframes1 = load_keyframes_from_folder(folder1)
keyframes2 = load_keyframes_from_folder(folder2)

# Vòng lặp cho các giá trị window size khác nhau
window_sizes = [1, 5, 10]
matching_results = {}

for window_size in window_sizes:
    matching_keyframes = exact_matching(keyframes1, keyframes2, window_size=window_size)
    matching_results[window_size] = matching_keyframes

# In kết quả cho từng window size
for ws, count in matching_results.items():
    print(f"Số lượng keyframe matching với window size {ws}: {count}")


Window size 10: 100%|██████████| 86/86 [00:59<00:00,  1.46frame/s]

Số lượng keyframe matching với window size 1: 60
Số lượng keyframe matching với window size 5: 62
Số lượng keyframe matching với window size 10: 62


In [10]:
import cv2
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def load_images_from_folder(folder_path):
    images = []
    image_files = sorted(os.listdir(folder_path))  # Đảm bảo các file được sắp xếp đúng thứ tự
    for filename in image_files:
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def exact_matching(frame1, frame2):
    # Kiểm tra kích thước của hai khung hình
    if frame1.shape != frame2.shape:
        return False

    # So sánh từng điểm ảnh
    return np.array_equal(frame1, frame2)

def compare_frame_pair(frame1, frame2):
    return exact_matching(frame1, frame2)

def compare_folders_exact(data_folder, gt_folder, num_threads=20):
    data_keyframes = load_images_from_folder(data_folder)
    gt_keyframes = load_images_from_folder(gt_folder)

    min_length = min(len(data_keyframes), len(gt_keyframes))
    total_frames = min_length

    # So sánh các cặp khung hình tương ứng bằng ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Tạo danh sách các tác vụ song song
        results = list(executor.map(compare_frame_pair, data_keyframes[:min_length], gt_keyframes[:min_length]))

    exact_matches = sum(results)
    matching_percentage = (exact_matches / total_frames) * 100 if total_frames > 0 else 0

    return {
        'Total Frames': total_frames,
        'Exact Matches': exact_matches,
        'Matching Percentage': matching_percentage
    }

# Ví dụ sử dụng
data_folder = 'data_based_scene_keyframe'
gt_folder = 'gt_keyframe'

results = compare_folders_exact(data_folder, gt_folder, num_threads=8)

print(f"Tổng số khung hình được so sánh: {results['Total Frames']}")
print(f"Số lượng khung hình khớp chính xác: {results['Exact Matches']}")
print(f"Phần trăm khớp chính xác: {results['Matching Percentage']:.2f}%")


Tổng số khung hình được so sánh: 86
Số lượng khung hình khớp chính xác: 0
Phần trăm khớp chính xác: 0.00%
